### Installing packages

In [1]:
#Instralling packages
!pip install natsort

### Defining ANSI codes for colored text prints 

In [17]:
# ANSI escape codes, to make log prints nicer
RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
BLUE = "\033[34m"
BOLD = "\033[1m"
ITALIC = "\x1B[3m"
UNDERLINED = "\033[4m"
RESET = "\033[0m"
WHITE_BG    = "\x1b[47m\033[30m" # adding \033[30m makes text black
GREEN_BG    = "\x1b[102m\033[30m" # adding \033[30m makes text black

# https://jakob-bagterp.github.io/colorist-for-python/ansi-escape-codes/standard-16-colors/#bright-colors_2

In [18]:
import pandas as pd
import os
from natsort import os_sorted
from datetime import datetime, timedelta
import requests
import zipfile
import warnings 
import urllib3

from dotenv import dotenv_values
from sqlalchemy import create_engine, types, text

# we'll suppress the "missing SSL certificate" warnings while downloading files
warnings.simplefilter("ignore", urllib3.exceptions.InsecureRequestWarning) 

## Data Download
**Sources**  
>Raw Data: https://transtats.bts.gov/PREZIP/  
>Website: https://transtats.bts.gov

In [19]:
# 1. Decide on starting month and total number of months
start = '07.2005' # Enter the starting month and the year (MM.YYYY)
length = 3 # How many months do you need?

In [20]:
# 2. Create folders for the zip files download and for the CSV-files extraction
os.makedirs('./downloads/extracted', exist_ok=True)

In [21]:
# 3. Create a list of months for the flight

# Generate list of MM.YYYY values for one year
def generate_year_list(start, length):
    start_date = datetime.strptime(start, '%m.%Y')
    return [f"{dt.year}_{dt.month}" for dt in
        (start_date + timedelta(days=31 * i) for i in range(length))]

# MM_YYYY values for the period lenght
year_month_list = generate_year_list(start, length)

print(year_month_list)

['2005_7', '2005_8', '2005_9']


In [22]:
# 4. Download ZIP files (~35 seconds per one file)

# Define the URL of the ZIP file
base_url = 'https://transtats.bts.gov/PREZIP/'
download_time = timedelta(0) # for time logging
disk_space_zip = 0

for year_month in year_month_list:

    # Define the URL of the ZIP file and the CSV file
    zip_name = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year_month}.zip'

    print(f'\n ⏳ This should take {RED}~35 seconds...{RESET}\n\n    ⬇️ {BLUE}downloading:{RESET} {zip_name}')
    print(f'    🐌 {YELLOW}wait for it...{RESET}', end='\r')
    start_time = datetime.now()

    # Send a HTTP request to the specified URL and save the response content
    response = requests.get(base_url+zip_name, verify=False) # we ignore the SSL certificate warnings

    with open(f'./downloads/{zip_name}', 'wb') as file: # save the ZIP in "downloads folder"
        file.write(response.content)
        print(f'    ✅ {GREEN}file saved:{RESET} {zip_name}', end=' ')
    
    # assessing the size of the downloaded file
    file_size = os.path.getsize(f'./downloads/{zip_name}') 
    size_in_mb = file_size / (1024 ** 2) 
    print(f'{GREEN}({size_in_mb:.2f} MB){RESET}\n')
    disk_space_zip += file_size

    # just some fun with basic time logging  
    end_time = datetime.now()
    time_difference = end_time - start_time
    download_time = download_time + time_difference
    if (time_difference.seconds // 60) < 1:
        print(f' 🦊 Actually it took: {YELLOW}{time_difference.seconds % 60} seconds\n{RESET}','-'*80)
    else:
        print(f' 🦊 Actually it took: {YELLOW}{time_difference.seconds // 60} minutes and {time_difference.seconds % 60} seconds\n{RESET}','-'*80)
print(f' 🦊 Total Download Time: {YELLOW}{download_time.seconds // 60} minutes and {download_time.seconds % 60} seconds\n{RESET}')
print(f' 🐹 Used Disk Space: {GREEN}({(disk_space_zip / (1024 ** 2)):.2f} MB){RESET}')


 ⏳ This should take ~35 seconds...

    ⬇️ downloading: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2005_7.zip
    ✅ file saved: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2005_7.zip (27.64 MB)

 🦊 Actually it took: 37 seconds
 --------------------------------------------------------------------------------

 ⏳ This should take ~35 seconds...

    ⬇️ downloading: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2005_8.zip
    ✅ file saved: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2005_8.zip (28.21 MB)

 🦊 Actually it took: 38 seconds
 --------------------------------------------------------------------------------

 ⏳ This should take ~35 seconds...

    ⬇️ downloading: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2005_9.zip
    ✅ file saved: On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2005_9.zip (24.82 MB)

 🦊 Actually it took: 32 seconds
 ----------------------------------------------------------

In [23]:
# 5. Extracting CSV files only

disk_space_csv = 0

for year_month in year_month_list:

    # Define the name of the ZIP file and the CSV file
    zip_name = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year_month}.zip'
    csv_name = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year_month}.csv'

    # Open the downloaded ZIP file
    with zipfile.ZipFile(f'./downloads/{zip_name}', 'r') as zip_ref:
        # Extract the CSV file
        zip_ref.extract(csv_name, path='./downloads/extracted/') # save the CSV in "downloads folder"
        print(f'    🍌 extracted "{csv_name}', end=' ')
        
    # assessing the size of the extracted file
    file_size = os.path.getsize(f'./downloads/extracted/{csv_name}') 
    size_in_mb = file_size / (1024 ** 2) 
    print(f"{GREEN}({size_in_mb:.2f} MB){RESET}\n")
    disk_space_csv += file_size

print('-'*80,f'\n 🐹 Used Disk Space: {GREEN}({(disk_space_csv / (1024 ** 2)):.2f} MB){RESET}')

    🍌 extracted "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_7.csv (269.35 MB)

    🍌 extracted "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_8.csv (270.04 MB)

    🍌 extracted "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_9.csv (244.96 MB)

-------------------------------------------------------------------------------- 
 🐹 Used Disk Space: (784.34 MB)


# Data Wrangling

### 1. adding all CSV file names to a list

In [24]:
# Add all file names from the "extracted" folder to a list
file_names = os.listdir('./downloads/extracted/')

# make sure only the data files are in the list
file_names_unordered = [fname for fname in file_names if fname.startswith("On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_")]

# using os_sorted function (from natsort) - able to sort strings with numbers ['2','1','11']
# sorted(['2','1','11']) # for comparison
data_files = os_sorted(file_names_unordered)

data_files

['On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_7.csv',
 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_8.csv',
 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_9.csv']

In [25]:
# we have 110 columns in each CSV...
file_check = pd.read_csv(f'./downloads/extracted/{data_files[0]}', low_memory = False)
file_check.shape

(627961, 110)

In [26]:
print(file_check.columns.to_list())

['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate', 'Reporting_Airline', 'DOT_ID_Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'Distance', 'DistanceGroup', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime'

In [27]:
# select columns to keep
def cols_to_keep(flights_raw):
    columns_to_keep = [
        "FlightDate",
        "DepTime",
        "CRSDepTime",
        "DepDelay",
        "ArrTime",
        "CRSArrTime",
        "ArrDelay",
        "Reporting_Airline",
        "Tail_Number",
        "Flight_Number_Reporting_Airline",
        "Origin",
        "Dest",
        "AirTime",
        "ActualElapsedTime",
        "Distance",
        "Cancelled",
        "Diverted",
    ]
    flights = flights_raw.loc[:, columns_to_keep]
    return flights

In [28]:
# rename columns
def rename_cols(flights):
    new_column_names = {
        'FlightDate': 'flight_date',
        'DepTime': 'dep_time',
        'CRSDepTime': 'sched_dep_time',
        'DepDelay': 'dep_delay',
        'ArrTime': 'arr_time',
        'CRSArrTime': 'sched_arr_time',
        'ArrDelay': 'arr_delay',
        'Reporting_Airline': 'airline',
        'Tail_Number': 'tail_number',
        'Flight_Number_Reporting_Airline': 'flight_number',
        'Origin': 'origin',
        'Dest': 'dest',
        'AirTime': 'air_time',
        'ActualElapsedTime': 'actual_elapsed_time',
        'Distance': 'distance',
        'Cancelled': 'cancelled',
        'Diverted': 'diverted'
    }
    flights.rename(columns=new_column_names, inplace=True)
    return flights

In [29]:
# change datatype
def change_dtypes(flights):
    types_change = {
        'flight_date': 'datetime64[ns]',
        'dep_time': 'Int16',
        'sched_dep_time': 'Int16',
        'dep_delay': 'Int16',
        'arr_time': 'Int16',
        'sched_arr_time': 'Int16',
        'arr_delay': 'Int16',
        'airline': 'O',
        'tail_number': 'O',
        'flight_number': 'Int16',
        'origin': 'O',
        'dest': 'O',
        'air_time': 'Int16',
        'actual_elapsed_time': 'Int16',
        'distance': 'Int16',
        'cancelled': 'Int16',
        'diverted': 'Int16'
    }
    flights = flights.astype(types_change)
    return flights

In [30]:
data_files


['On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_7.csv',
 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_8.csv',
 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_9.csv']

In [31]:
# list for separate dataframes
flights_list = []

#  loop over the extracted csv files and execute functions 
for file in data_files:
    print(file)
    # 1. read as a dataframe
    print('reading...', end=" ")
    flights_raw = pd.read_csv(f'./downloads/extracted/{file}', low_memory = False) 

    # 2.select columns to keep
    flights_select = cols_to_keep(flights_raw) 
    print('filter colums...', end=" ")
    
    # 3. rename columns
    flights_rename = rename_cols(flights_select) 
    print('rename colums...', end=" ")

    # 4. change data types
    flights_dtypes = change_dtypes(flights_rename) 
    print('change dtypes...', end=" ")
    
    # 5. add to the list of dateframes
    flights_list.append(flights_dtypes) 
    print(f'✅ {GREEN}appended to flight_list{RESET}\n')
    
print(f'Done. The list has {len(flights_list)} elements')

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_7.csv
reading... filter colums... rename colums... change dtypes... ✅ appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_8.csv
reading... filter colums... rename colums... change dtypes... ✅ appended to flight_list

On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2005_9.csv
reading... filter colums... rename colums... change dtypes... ✅ appended to flight_list

Done. The list has 3 elements


In [32]:
# 6. concatenate the list of dataframes to a one dataframe
flights_all = pd.concat(flights_list)

In [33]:
# sort dataframe 
flights_all.sort_values(['flight_date','sched_dep_time'], inplace=True)
flights_all

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,actual_elapsed_time,distance,cancelled,diverted
65173,2005-07-01,55,5,50,759,711,48,AS,N647AS,168,ANC,DEN,284,304,2405,0,0
123803,2005-07-01,222,8,134,1001,755,126,DL,N6705Y,2093,LAS,JFK,252,279,2248,0,0
59482,2005-07-01,22,10,12,504,502,2,AA,N3CWAA,2482,LAX,DFW,140,162,1235,0,0
109493,2005-07-01,201,10,111,919,740,99,DH,N807BR,777,LAS,IAD,229,258,2066,0,0
305809,2005-07-01,15,10,5,717,732,-15,NW,N551NW,336,LAX,DTW,222,242,1979,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224891,2005-09-30,3,2359,4,138,126,12,HP,N675AW,374,LAS,OAK,59,95,407,0,0
224892,2005-09-30,2351,2359,-8,51,100,-9,HP,N624AW,636,LAS,ONT,32,60,197,0,0
224902,2005-09-30,25,2359,26,127,111,16,HP,N907AW,57,LAS,PHX,39,62,256,0,0
224915,2005-09-30,25,2359,26,139,106,33,HP,N807AW,730,LAS,SAN,45,74,258,0,0


In [34]:
# countercheck the time period
flights_all['flight_date'].min(), flights_all['flight_date'].max()

(Timestamp('2005-07-01 00:00:00'), Timestamp('2005-09-30 00:00:00'))

In [35]:
# define the file name for the combined CSV file (using period's first and last month)
output_file_name = f'flights_from_{year_month_list[0]}_until_{year_month_list[-1]}.csv'
output_file_name

'flights_from_2005_7_until_2005_9.csv'

In [36]:
# create folder 'data'
os.makedirs('./data', exist_ok=True)

In [37]:
# saving
flights_all.to_csv(f'./data/{output_file_name}', index=False)

print(f' ✅ {GREEN}Combined Dataset Saved:{RESET} {output_file_name}', end=' ')

# assessing the size of the extracted file
file_size = os.path.getsize(f'./data/{output_file_name}') 
size_in_mb = file_size / (1024 ** 2) 
print(f"{GREEN}({size_in_mb:.2f} MB){RESET}\n")

# Get the absolute path
absolute_path = os.path.abspath(f'./data/{output_file_name}')
print(absolute_path)


 ✅ Combined Dataset Saved: flights_from_2005_7_until_2005_9.csv (127.52 MB)

/Users/pavelpavlov/Desktop/gaussian_peppers/below_zero/pavel/data/flights_from_2005_7_until_2005_9.csv
